# Su Doku - Problem 96
<p>Su Doku (Japanese meaning <i>number place</i>) is the name given to a popular puzzle concept. Its origin is unclear, but credit must be attributed to Leonhard Euler who invented a similar, and much more difficult, puzzle idea called Latin Squares. The objective of Su Doku puzzles, however, is to replace the blanks (or zeros) in a 9 by 9 grid in such that each row, column, and 3 by 3 box contains each of the digits 1 to 9. Below is an example of a typical starting puzzle grid and its solution grid.</p>
<div class="center">
<img src="project/images/p096_1.png" alt="p096_1.png" />     <img src="project/images/p096_2.png" alt="p096_2.png" /></div>
<p>A well constructed Su Doku puzzle has a unique solution and can be solved by logic, although it may be necessary to employ "guess and test" methods in order to eliminate options (there is much contested opinion over this). The complexity of the search determines the difficulty of the puzzle; the example above is considered <i>easy</i> because it can be solved by straight forward direct deduction.</p>
<p>The 6K text file, <a href="project/resources/p096_sudoku.txt">sudoku.txt</a> (right click and 'Save Link/Target As...'), contains fifty different Su Doku puzzles ranging in difficulty, but all with unique solutions (the first puzzle in the file is the example above).</p>
<p>By solving all fifty puzzles find the sum of the 3-digit numbers found in the top left corner of each solution grid; for example, 483 is the 3-digit number found in the top left corner of the solution grid above.</p>

## Solution.

In [17]:
from collections import defaultdict

In [63]:
with open('data_96.txt', 'r') as file:
    s = file.read()

s = s.split('\n')
data = defaultdict(list)
i = 0
n = 0
while i < len(s):
    while i < len(s) and 'Grid' not in s[i]:
        data[n].append([int(x) for x in s[i]])
        i += 1
    n += 1
    i += 1
        

In [64]:
def can_put(k, i, j, board):
    '''
    Check if one can put k in the cell board[i][j] without causing any contradictions.
    '''
    if k in board[i]:
        return False

    for x in range(9):
        if board[x][j] == k:
            return False

    n = i // 3
    m = j // 3
    for x in range(3 * n, 3 * n + 3):
        for y in range(3 * m, 3 * m + 3):
            if board[x][y] == k:
                return False

    return True

In [65]:
def solve_sudoku(board):
    def solve():
        for i in range(9):
            for j in range(9):
                if board[i][j] == 0:
                    for num in range(1, 10):
                        if can_put(num, i, j, board):
                            board[i][j] = num
                            if solve():
                                return True
                            board[i][j] = 0
                    return False
        return True

    solve()
    return board


In [66]:
ans = 0
for i in data:
    t = solve_sudoku(data[i])
    ans += 100 * t[0][0] + 10 * t[0][1] + t[0][2]

print(ans)

24702


### More efficient.

In [68]:
with open('data_96.txt', 'r') as file:
    s = file.read()

s = s.split('\n')
data = defaultdict(list)
i = 0
n = 0
while i < len(s):
    while i < len(s) and 'Grid' not in s[i]:
        data[n].append([int(x) for x in s[i]])
        i += 1
    n += 1
    i += 1

def can_put(num, row, col, board, rows, cols, boxes):
    '''
    Check if one can put num in the cell board[row][col] without causing any contradictions.
    '''
    box_index = (row // 3) * 3 + (col // 3)
    if num in rows[row] or num in cols[col] or num in boxes[box_index]:
        return False
    return True

def solve_sudoku(board):
    rows = [set() for _ in range(9)]
    cols = [set() for _ in range(9)]
    boxes = [set() for _ in range(9)]
    empty = []

    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                empty.append((i, j))
            else:
                num = board[i][j]
                rows[i].add(num)
                cols[j].add(num)
                box_index = (i // 3) * 3 + (j // 3)
                boxes[box_index].add(num)

    def solve():
        if not empty:
            return True

        row, col = empty.pop()
        box_index = (row // 3) * 3 + (col // 3)

        for num in range(1, 10):
            if can_put(num, row, col, board, rows, cols, boxes):
                board[row][col] = num
                rows[row].add(num)
                cols[col].add(num)
                boxes[box_index].add(num)

                if solve():
                    return True

                board[row][col] = 0
                rows[row].remove(num)
                cols[col].remove(num)
                boxes[box_index].remove(num)

        empty.append((row, col))
        return False

    solve()
    return board

ans = 0
for i in data:
    t = solve_sudoku(data[i])
    ans += 100 * t[0][0] + 10 * t[0][1] + t[0][2]

print(ans)

24702
